In [1]:
#installing stanza
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 49.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [2]:
import stanza

In [3]:
#download the language model
stanza.download("en")

# Create the pipeline, specifying the language:
nlp = stanza.Pipeline(lang="en", processors='tokenize,mwt,ner')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| ner       | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


clone the file, create a path to tsv

In [4]:
# clone our FASDH25-portfolio2 folder here:
!git clone https://github.com/faizahmed-22/FASDH25-portfolio2.git

Cloning into 'FASDH25-portfolio2'...
remote: Enumerating objects: 4389, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 4389 (delta 12), reused 3 (delta 3), pack-reused 4367 (from 1)
Receiving objects: 100% (4389/4389), 17.94 MiB | 17.42 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [ ]:
import requests #import requests library to interact with GeoNames
import time #import time library to carry out time related functions

geonames_username = "_faizahmed_" #making the username a variable

#defining a function get_coordinates and setting place as an input to try and find geographical coordinates
def get_coordinates(place, username=geonames_username, fuzzy=0, timeout=1):
    # wait a short while, so that we don't overload the server:
    time.sleep(timeout)
    # make the API call:
    url = "http://api.geonames.org/searchJSON?"
    params = {
        "q": place,
        "username": username,
        "fuzzy": fuzzy,
        "maxRows": 1,
        "isNameRequired": True
        }
    response = requests.get(url, params=params) #send the request to the geonames server and store the response
    # convert the response into a dictionary:
    results = response.json() #converts the response from json format to a pyhton dictionary
    print(results)

    # get the first result:
    try: #extracting latitude and longitude from the results
        result = results["geonames"][0]
        return {"latitude": result["lat"], "longitude": result["lng"]}
    except (IndexError, KeyError): #what will happen if no results are found
        print("No results found for your API call", response.request.url)

import csv

filename="ner_gazetteer.tsv"



with open(filename, mode="w", encoding="utf-8") as file: #open the file for writing
    header = "place\tlatitude\tlongitude\n" #creating a header row in the file
    file.write(header)

    with open("ner_counts.tsv", encoding="utf-8") as infile:
        reader = csv.DictReader(infile, delimiter='\t')

        for row in reader:
            place_name = row['place']
            #Pass place_name as the 'place' argument to the get_coordinates function
            coordinates = get_coordinates(place_name) #calls the function to find the coordinates for each place name
            if coordinates: #if the coordinates are found they will be written in the output file
                latitude = coordinates["latitude"] #finds the latitude and writes it in the output file
                longitude = coordinates["longitude"] #finds the longitude and writes it in the output file
                row_data = f"{place_name}\t{latitude}\t{longitude}\n" #formats the data and separates the values that will be written in the tsv file in a single row
                file.write(row_data)
            else: #if the coordinates are not found it will write NA in the output file
                row_data = f"{place_name}\tNA\tNA\n" #it formats the string containing NA for both latitude and longitude and separates them
                file.write(row_data)


{'totalResultsCount': 33, 'geonames': [{'adminCode1': '00', 'lng': '34.75', 'geonameId': 294640, 'toponymName': 'State of Israel', 'countryId': '294640', 'fcl': 'A', 'population': 8883800, 'countryCode': 'IL', 'name': 'Israel', 'fclName': 'country, state, region,...', 'countryName': 'Israel', 'fcodeName': 'independent political entity', 'adminName1': '', 'lat': '31.5', 'fcode': 'PCLI'}]}
{'totalResultsCount': 40, 'geonames': [{'adminCode1': 'GZ', 'lng': '34.46672', 'geonameId': 281133, 'toponymName': 'Gaza', 'countryId': '6254930', 'fcl': 'P', 'population': 410000, 'countryCode': 'PS', 'name': 'Gaza', 'fclName': 'city, village,...', 'adminCodes1': {}, 'countryName': 'Palestine', 'fcodeName': 'seat of a first-order administrative division', 'adminName1': 'Gaza Strip', 'lat': '31.50161', 'fcode': 'PPLA'}]}
{'totalResultsCount': 49, 'geonames': [{'adminCode1': '00', 'lng': '35.20329', 'geonameId': 6254930, 'toponymName': 'Palestine', 'countryId': '6254930', 'fcl': 'A', 'population': 45690